In [1]:
import pyspark
print(pyspark.__version__)

#!pip install elasticsearch
#!pip install python-dotenv

3.3.2


In [2]:
import os
os.environ[
    "PYSPARK_SUBMIT_ARGS"
] = "--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.2,org.elasticsearch:elasticsearch-spark-30_2.12:7.16.2,com.fasterxml.jackson.module:jackson-module-scala_2.12:2.13.0 pyspark-shell"

In [3]:
from pyspark.sql import SparkSession
# create a Spark session
spark = SparkSession.builder.appName("kafka_elastic_test").getOrCreate()

In [4]:
# create a Kafka stream for transaction
df_transaction_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "dbserver1.fineract_default.m_savings_account_transaction") \
    .option("group.id", "1") \
    .load()

# create a Kafka stream for account
df_account_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "dbserver1.fineract_default.m_savings_account") \
    .load()

In [5]:
from pyspark.sql.types import StructType, StructField, LongType, DoubleType, StringType

# Define the schema for the DataFrame
schema_transaction = StructType([
    StructField("account-id", LongType(), True),
    StructField("amount", DoubleType(), True),
    StructField("customer-id", LongType(), True),
    StructField("datetime", StringType(), True),
    StructField("is_fraud", StringType(), True),
    StructField("transaction-id", LongType(), True),
    StructField("type", StringType(), True),
    StructField("@timestamp", StringType(), True)
])

# Define the schema for the DataFrame
schema_account = StructType([
    StructField("customer_id",  LongType(), True),
    StructField("account_id", LongType(), True)
])

account_df = spark.createDataFrame([(-1,-1),], schema=schema_account)

dic_trans = {}

In [6]:
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import RequestError
import os
from dotenv import load_dotenv

# Define the Elasticsearch index name
es_index = "transactions_index"
es_port = 9200
es_host = "elasticsearch"


# Load environment variables from .env file
load_dotenv()

# Access the username and password
username = os.environ.get("USERNAME")
password = os.environ.get("PASSWORD")

# Create an Elasticsearch client
es = Elasticsearch(
    [{"host": es_host, "port": es_port, "scheme": "http"}],
     basic_auth=(username, password)
)

# Create a mapping for the Elasticsearch index
# Define the index mapping
mapping = {
    "mappings": {
        "properties": {
            "@timestamp": {"type": "date"},
            "account-id": {"type": "long"},
            "amount": {"type": "double"},
            "customer-id": {"type": "long"},
            "datetime": {"type": "date", "format": "yyyy-MM-dd HH:mm:ss"},
            "is_fraud": {"type": "keyword"},
            "transaction-id": {"type": "long"},
            "type": {"type": "keyword"},
        }
    }
}

# Check if the index exists, and create it if it does not
if not es.indices.exists(index=es_index):
    try:
        # Create the Elasticsearch index if it doesn't exist
        es.indices.create(index=es_index, body=mapping)
    except RequestError as e:
        print(f"Index creation failed: {e}")
        exit(1)
    print(f"Created Elasticsearch index '{es_index}'")
    
# Write the streaming DataFrame to Elasticsearch
def write_to_es(es_df):
    es_df.show()
    es_df.write \
    .format("org.elasticsearch.spark.sql") \
    .option("es.nodes", es_host) \
    .option("es.port", es_port) \
    .option("es.net.http.auth.user", username) \
    .option("es.net.http.auth.pass", password) \
    .option("es.resource", es_index) \
    .mode("append") \
    .save()

In [7]:
from datetime import datetime as dt

def is_night(datetime_str):
    datetime_format = "%Y-%m-%d %H:%M:%S"
    datetime_obj = dt.strptime(datetime_str, datetime_format)
    hour = datetime_obj.hour
    
    if hour >= 22 or hour < 6:
        return 1
    else:
        return 0

def is_weekend(datetime_str):
    datetime_format = "%Y-%m-%d %H:%M:%S"
    datetime_obj = dt.strptime(datetime_str, datetime_format)
    weekday = datetime_obj.weekday()
    
    if weekday >= 5:  # 5 and 6 correspond to Saturday and Sunday
        return 1
    else:
        return 0

In [8]:
import redis
import datetime

# Create a Redis connection
redis_host = "redis"
redis_port = 6379
redis_db_tr = 0
redis_db_acc = 1

redis_client = redis.Redis(host=redis_host, port=redis_port, db= redis_db_tr)

redis_client_acc = redis.Redis(host=redis_host, port=redis_port, db= redis_db_acc)

In [9]:
import json

def add_transaction_to_history(key, data):
        # Serialize the dictionary into a JSON string
        data_json = json.dumps(data)

        # Save the serialized data in Redis
        redis_client.set(key, data_json)
        
        # Set the expiration time for the key
        redis_client.expire(key, 7889229) # 3 months
        
        
def add_account_to_cache(key, data):
        # Serialize the dictionary into a JSON string
        data_json = json.dumps(data)

        # Save the serialized data in Redis
        redis_client_acc.set(key, data_json)
        
        # Set the expiration time for the key
        redis_client_acc.expire(key, 604800) # 1 week

In [10]:
def get_transactions_history():
    
    # Get all keys in Redis
    all_keys = redis_client.keys("*")

    # Retrieve values for each key
    data = {}
    for key in all_keys:
        value = redis_client.get(key)
        data[key.decode("utf-8")] = json.loads(value.decode("utf-8"))

    # Convert list of dictionaries to RDD
    rdd = spark.sparkContext.parallelize(list(data.values()))

    # Create DataFrame from RDD
    return spark.createDataFrame(rdd)

def get_account_from_cache(key):
    
    data = {}
    # Retrieve values for each key
    value = redis_client_acc.get(key)
    data[key] = json.loads(value.decode("utf-8"))

    # Convert list of dictionaries to RDD
    rdd = spark.sparkContext.parallelize(list(data.values()))

    # Create DataFrame from RDD
    return spark.createDataFrame(rdd)

In [11]:
from pyspark.sql.functions import col, avg, count, current_date, date_sub
from pyspark.sql import Window

def get_ml_transaction_input_byIds(account_id, customer_id):
    dict_ml = {}
    
    # get transaction from redis
    trans_df = get_transactions_history()

    # Filter transactions for the given customer_id
    customer_filtered_df = trans_df.filter(col("customer_id") == customer_id)

    # Calculate customer_id average amount for different time periods
    dict_ml['customer_id_avrge_amount_1day'] = customer_filtered_df.filter(col("datetime") >= date_sub(current_date(), 1)).select(avg("amount")).collect()[0][0]
    dict_ml['customer_id_avrge_amount_1week'] = customer_filtered_df.filter(col("datetime") >= date_sub(current_date(), 7)).select(avg("amount")).collect()[0][0]
    dict_ml['customer_id_avrge_amount_1month'] = customer_filtered_df.filter(col("datetime") >= date_sub(current_date(), 30)).select(avg("amount")).collect()[0][0]
    dict_ml['customer_id_avrge_amount_3month'] = customer_filtered_df.filter(col("datetime") >= date_sub(current_date(), 90)).select(avg("amount")).collect()[0][0]

    # Calculate customer_id transaction counts for different time periods
    dict_ml['customer_id_count_1day'] = customer_filtered_df.filter(col("datetime") >= date_sub(current_date(), 1)).count()
    dict_ml['customer_id_count_1week'] = customer_filtered_df.filter(col("datetime") >= date_sub(current_date(), 7)).count()
    dict_ml['customer_id_count_1month'] = customer_filtered_df.filter(col("datetime") >= date_sub(current_date(), 30)).count()
    dict_ml['customer_id_count_3month'] = customer_filtered_df.filter(col("datetime") >= date_sub(current_date(), 90)).count()

    # Filter transactions for the given account_id
    account_filtered_df = trans_df.filter(col("account_id") == account_id)

    # Calculate account_id average amount for different time periods
    dict_ml['account_id_avrge_amount_1day'] = account_filtered_df.filter(col("datetime") >= date_sub(current_date(), 1)).select(avg("amount")).collect()[0][0]
    dict_ml['account_id_avrge_amount_1week'] = account_filtered_df.filter(col("datetime") >= date_sub(current_date(), 7)).select(avg("amount")).collect()[0][0]
    dict_ml['account_id_avrge_amount_1month'] = account_filtered_df.filter(col("datetime") >= date_sub(current_date(), 30)).select(avg("amount")).collect()[0][0]
    dict_ml['account_id_avrge_amount_3month'] = account_filtered_df.filter(col("datetime") >= date_sub(current_date(), 90)).select(avg("amount")).collect()[0][0]

    # Calculate account_id transaction counts for different time periods
    dict_ml['account_id_count_1day'] = account_filtered_df.filter(col("datetime") >= date_sub(current_date(), 1)).count()
    dict_ml['account_id_count_1week'] = account_filtered_df.filter(col("datetime") >= date_sub(current_date(), 7)).count()
    dict_ml['account_id_count_1month'] = account_filtered_df.filter(col("datetime") >= date_sub(current_date(), 30)).count()
    dict_ml['account_id_count_3month'] = account_filtered_df.filter(col("datetime") >= date_sub(current_date(), 90)).count()
    
    return dict_ml

In [12]:
# mllib ia model

from pyspark.ml.classification import LogisticRegressionModel

lr_model = LogisticRegressionModel.load("lr_ml")

In [13]:
%%time
from pyspark.ml.linalg import Vectors

def predict_isfraude(dic_trans):
    
    # get input variable for ml
    ml_input_var = get_ml_transaction_input_byIds( dic_trans['account_id'], dic_trans['customer_id'])
    
    # create the feature vector
    dense_vector = Vectors.dense([
        dic_trans['amount'],
        ml_input_var['customer_id_avrge_amount_1day'],
        ml_input_var['customer_id_avrge_amount_1week'], 
        ml_input_var['customer_id_avrge_amount_1month'],
        ml_input_var['customer_id_avrge_amount_3month'],
        ml_input_var['customer_id_count_1day'],
        ml_input_var['customer_id_count_1week'],
        ml_input_var['customer_id_count_1month'],
        ml_input_var['customer_id_count_3month'],
        ml_input_var['account_id_avrge_amount_1day'],
        ml_input_var['account_id_avrge_amount_1week'],
        ml_input_var['account_id_avrge_amount_1month'],
        ml_input_var['account_id_avrge_amount_3month'],
        ml_input_var['account_id_count_1day'],
        ml_input_var['account_id_count_1week'],
        ml_input_var['account_id_count_1month'],
        ml_input_var['account_id_count_3month'],
        dic_trans['in_weekend'], dic_trans['at_night']
    ])
    data_vect = spark.createDataFrame([(dense_vector, ), ], ['features'])
    is_fraude = lr_model.transform(data_vect).select(['prediction']).collect()[0][0]
    
    return "valid" if is_fraude == 0.0 else "fraudulent"

CPU times: user 14 µs, sys: 3 µs, total: 17 µs
Wall time: 20.3 µs


In [14]:
from pyspark.sql.functions import from_unixtime, date_format
from pyspark.sql.functions import col

import json
import datetime
import base64
import decimal
from time import sleep

def getDecimalFromKafka(encoded):
    
    # Decode the Base64 encoded string and create a BigInteger from it
    decoded = decimal.Decimal(int.from_bytes(base64.b64decode(encoded), byteorder='big', signed=False))

    # Create a context object with the specified scale
    context = decimal.Context(prec=28, rounding=decimal.ROUND_HALF_DOWN)

    # Set the scale of the decimal value using the context object
    decimal_value = decoded.quantize(decimal.Decimal('.1') ** 3, context=context)

    return decimal_value/1000000

def write_to_es_transaction(df_t, epoch_id):
    
    global account_df
    
    row_transactions = df_t.collect()
    
    for row_transaction in row_transactions:
    
            value_dict_transaction = json.loads(row_transaction.value)
            
            if value_dict_transaction['payload']['before'] == None :
                
                timestamp = value_dict_transaction['payload']['after']['created_date']/1000
                # convert Unix timestamp to a datetime object
                dt = datetime.datetime.fromtimestamp(timestamp)
                # format datetime object as "yyyy-mm-dd hh:mm:ss"
                dic_trans['datetime'] = dt.strftime("%Y-%m-%d %H:%M:%S")
                formatted_date_es = dt.strftime("%Y-%m-%dT%H:%M:%S.%f%z")
        
                dic_trans['account_id'] = value_dict_transaction['payload']['after']['savings_account_id']
        
                while account_df.filter(col("account_id") == dic_trans['account_id']).count() == 0:
                    # Wait for 0.01 second before checking again
                    sleep(0.01)
                    account_df = get_account_from_cache(dic_trans['account_id'])
        
                # Code to execute after the condition becomes true
                # Filter the DataFrame to get rows where "account-id" is in account_df["account-id"]
                filtered_account_df = account_df.filter(account_df["account_id"] == dic_trans['account_id'])
                # Select the "customer-id" column from the filtered DataFrame
                dic_trans['customer_id'] = filtered_account_df.select("customer_id").collect()[0][0]
                
                op_type = "DEBIT" if value_dict_transaction['payload']['after']['transaction_type_enum'] == 2 else "CREDIT"  
                
                dic_trans['amount'] = float(getDecimalFromKafka(value_dict_transaction['payload']['after']['amount']))
                
                # save this transacton in redis
                add_transaction_to_history(value_dict_transaction['payload']['after']['id'], dic_trans)
                
                dic_trans['in_weekend'] = is_night(dic_trans['datetime'])
                dic_trans['at_night'] = is_weekend(dic_trans['datetime'])
                
                new_row_transaction = spark.createDataFrame([(dic_trans['account_id'],
                                                      dic_trans['amount'],
                                                      dic_trans['customer_id'],
                                                      dic_trans['datetime'],
                                                      predict_isfraude(dic_trans),#-----test after will be predected by ML
                                                      value_dict_transaction['payload']['after']['id'],
                                                      op_type,
                                                      formatted_date_es,
                                                     )], schema=schema_transaction)
                        
                write_to_es(new_row_transaction)
                    
        
def write_to_es_account(df_a, epoch_id):
    global account_df
        
    row_accounts = df_a.collect()
    
    for row_account in row_accounts :
    
            dict_account = {}
            value_dict_account = json.loads(row_account.value)
            dict_account['customer_id'] = value_dict_account['payload']['after']['client_id']
            dict_account['account_id'] = value_dict_account['payload']['after']['id']
        
            # add account to cache
            add_account_to_cache(dict_account['account_id'], dict_account)


In [ ]:
# Call the write_to_es function on each micro-batch of data
value_df_account = df_account_stream.selectExpr("CAST(value AS STRING)")
query_account = value_df_account.writeStream.foreachBatch(write_to_es_account).start()

value_df_transaction = df_transaction_stream.selectExpr("CAST(value AS STRING)")
query_transaction = value_df_transaction.writeStream.foreachBatch(write_to_es_transaction).start()

# Wait for the stream to finish
#query_account.awaitTermination()
query_transaction.awaitTermination()

+----------+------+-----------+-------------------+--------+--------------+------+--------------------+
|account-id|amount|customer-id|           datetime|is_fraud|transaction-id|  type|          @timestamp|
+----------+------+-----------+-------------------+--------+--------------+------+--------------------+
|         4| 300.0|          2|2023-05-25 00:38:13|   valid|           148|CREDIT|2023-05-25T00:38:...|
+----------+------+-----------+-------------------+--------+--------------+------+--------------------+

+----------+------+-----------+-------------------+--------+--------------+------+--------------------+
|account-id|amount|customer-id|           datetime|is_fraud|transaction-id|  type|          @timestamp|
+----------+------+-----------+-------------------+--------+--------------+------+--------------------+
|         4|  33.0|          2|2023-05-25 00:38:51|   valid|           149|CREDIT|2023-05-25T00:38:...|
+----------+------+-----------+-------------------+--------+---